<div class="alert alert-success" style="font-size: 16px;">
    <div style="font-size: 22px; font-weight:bold; margin: 10px 0px;">Topic Modeling</div>
After preprocessing we have to vectorize our preprocessed text and train our topic model.
</div>

## Imports

In [69]:
#!py -m spacy download en_core_web_sm

from tqdm.notebook import tqdm
import spacy

from conferences import get_conference_data

try:
    nlp
except:
    nlp = spacy.load("nl_core_news_sm")

## Get Data

In [3]:
conferences_data = get_conference_data()

In [4]:
text_by_Rutte = [conference['text'] for conference in conferences_data[0]]
text_by_de_jonge = [conference['text'] for conference in conferences_data[1]]

merged_texts = [text_by_Rutte, text_by_de_jonge]
merged_texts = [item for sublist in merged_texts for item in sublist]

## Tokenize Texts

In [44]:
%%time
processed_texts = [text for text in tqdm(nlp.pipe(merged_texts, n_process=1, disable=["ner", "parser"]), total=len(merged_texts))] 

  0%|          | 0/58 [00:00<?, ?it/s]

Wall time: 54.5 s


In [67]:
tokenized_texts_sw = [[word.text.lower() for word in processed_text if not word.is_stop] 
                      for processed_text in processed_texts]
print(tokenized_texts_sw)

[['goedenavond', '.', 'vandaag', '1', 'september', '.', 'ongetwijfeld', 'weet', ':', 'laatste', 'formele', 'datum', 'routekaart', 'coronamaatregelen', '.', 'directe', 'aanleiding', 'persconferentie', '.', 'zeg', 'meteen', ':', 'groot', 'onverwacht', 'nieuws', 'vandaag', '.', 'denken', 'goed', 'stand', 'zaken', 'bespreken', ',', 'natuurlijk', 'laatste', 'persconferentie', 'twee', 'weken', 'nodige', 'gebeurd', '.', 'willen', 'vandaag', 'twee', 'vragen', 'doornemen', '.', 'duiding', ':', 'staan', ',', 'half', 'jaar', 'begin', 'uitbraak', '?', 'beoordelen', 'recente', 'cijfers', '?', 'kort', 'zeggen', '.', 'tweede', ',', 'komende', 'tijd', 'tweede', 'golf', 'blijven', 'voorbereid', '?', 'hugo', 'jonge', 'meteen', 'ingaan', '.', 'cijfers', 'induik', ',', 'beginnen', 'keer', 'duidelijk', 'gezegd', 'uiteraard', 'iedereen', 'vragen', 'stellen', 'kabinet', 'kritiek', '.', 'sterker', ':', 'gek', 'gebeuren', '.', 'volkomen', 'logisch', 'mensen', 'half', 'jaar', 'beperkingen', 'onderzoek', 'uitgaa

## Vectorization

In [72]:
from gensim.corpora import Dictionary

MIN_DF = 1 # minium document frequency
MAX_DF = 0.6 # maximum document frequency

dictionary = Dictionary(tokenized_texts_sw) # get the vocabulary
dictionary.filter_extremes(no_below=MIN_DF, 
                           no_above=MAX_DF)
corpus = [dictionary.doc2bow(text) for text in tokenized_texts_sw]

## Train Model

In [ ]:
from gensim.models.wrappers import LdaMallet

PATH_TO_MALLET = ''

N_TOPICS = 2
N_ITERATIONS = 1000 # usually 1000 will do

lda = LdaMallet(PATH_TO_MALLET,
                corpus=corpus,
                id2word=dictionary,
                num_topics=N_TOPICS,
                optimize_interval=10,
                iterations=N_ITERATIONS)

## Show Topics

In [ ]:
for topic in range(N_TOPICS):
    words = lda.show_topic(topic, 10)
    topic_n_words = ' '.join([word[0] for word in words])
    print('Topic {}: {}'.format(str(topic), topic_n_words))

## Visualisation with pyLDAvis

In [ ]:
import gensim

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

pyLDAvis.enable_notebook()

lda_conv = gensim.models.wrappers.ldamallet.malletmodel2ldamodel(lda)

gensimvis.prepare(lda_conv, corpus, dictionary)

In [ ]:
# TEST

texts = ['Image this a not a list of two texts', 'But like millions!']
dutch_texts = ['Dus dat weet u eigenlijk nog niet.','En u weet eigenlijk ook niet hoeveel beschermingsmiddelen u dan nodig heeft?']
processed_texts2 = [text for text in nlp2.pipe(texts, 
                                              n_process=1, disable=["ner",
                                                       "parser"])]

processed_texts2
for text in processed_texts2:
    for token in text:
        print(token.text, token.pos_, token.dep_)

([{'date': '2020-09-01',
   'text': 'Ja goedenavond. Het is vandaag 1 september. En zoals u ongetwijfeld nog weet: de laatste formele datum op onze routekaart van coronamaatregelen. Dat is de directe aanleiding voor deze persconferentie. Ik zeg maar meteen: we hebben geen groot of onverwacht nieuws vandaag. Toch denken we dat het goed is de stand van zaken met u te bespreken, want er is natuurlijk na de laatste persconferentie van twee weken geleden wel het nodige gebeurd. We willen vandaag twee vragen met u doornemen. Ten eerste de duiding: waar staan we nu, ongeveer een half jaar na het begin van de uitbraak? Hoe beoordelen we de recente cijfers? Daar zal ik kort iets over zeggen. Ten tweede, wat doen we nu en de komende tijd om die tweede golf voor te blijven en er toch op voorbereid te zijn? Daar zal Hugo de Jonge zo meteen op ingaan. Maar voordat ik de cijfers induik, wil ik om te beginnen een keer duidelijk gezegd hebben dat uiteraard iedereen vragen mag stellen bij wat we als ka